# House sale price prediction
In this project, we'll work with housing data for the city of Ames, Iowa, United States, from 2006 to 2010. Information on the data set can be found [here](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627). The prediction models will be based on linear regressions.

In [68]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [69]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
print("The data frame has " + str(len(df.columns)) + " columns and " + str(len(df)) + " rows."  )
df.dtypes.value_counts()

The data frame has 82 columns and 2930 rows.


object     43
int64      28
float64    11
dtype: int64

We'll begin by creating a function, `train_and_test()`, which takes three inputs:
- a data frame
- the columns to be selected for the model, including the target column
- the target column ('SalePrice' by default)

The function will:
- select the columns for the model
- split the data frame into a training set and a test set
- instantiate and train a linear regression model using only numeric columns as variables
- predict the target variable on the test set
- return the rooted mean squared error (`rmse`) between predicted and actual values

In [70]:
def train_and_test(df,
                   features,
                   target = 'SalePrice'):  
        
    # Feature selection
    df = df[features]
    
    # Train and test data split
    train = df[:int(len(df)/2)]
    test = df[int(len(df)/2):]
             
    # Selection of numeric columns
    train = train.select_dtypes(include=['integer', 'float'])
    test = test.select_dtypes(include=['integer', 'float'])
    
    # Model
    features = train.columns.drop(target)
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

rmse = train_and_test(df,["Gr Liv Area", "SalePrice"])

rmse

57120.50729008638

We will transform (function `transform_features()`) each numeric column with missing values by filling in with the most common value in that column.

In [71]:
def transform_features(df):
    vals = df.mode(numeric_only=True)
    vals = vals.iloc[0].to_dict()
    df = df.fillna(vals)
    return  df

We will create new features, based on existing features, that better represent information relevant for the prediction.

In [72]:
def transform_features(df):
    vals = df.mode(numeric_only=True)
    vals = vals.iloc[0].to_dict()
    df = df.fillna(vals)
    
    # New features
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']
    
    # Removal of rows with negative values for both of these new features
    indexes_to_remove = (df['Years Before Sale']<0) | (df['Years Since Remod']<0)
    indexes_to_remove = df[indexes_to_remove].index
    df = df.drop(indexes_to_remove)
            
    return  df
len(transform_features(df))

2927

We will drop columns (function `remove_features()`) that:
- aren't useful as predictors ('PID' and 'Order')
- leak data about the final sale ('Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold', information about each column can be found [here](https://s3.amazonaws.com/dq-content/307/data_description.txt))

In [73]:
def remove_features(df,
                    features_to_remove=["PID", "Order","Mo Sold", 
                                        "Sale Condition", "Sale Type", 
                                        "Yr Sold","Year Built", 
                                        "Year Remod/Add" ]):
    df = df.drop(features_to_remove,axis=1)
    return  df
len(remove_features(df).columns)

74

We will update the `remove_features()` function so as to drop:
- any column with 5% or more missing values
- any text columns with 1 or more missing values

In [74]:
def remove_features(df,features_to_remove=["PID", "Order","Mo Sold", "Sale Condition", "Sale Type", "Yr Sold" ]):
    df = df.drop(features_to_remove,axis=1)
    
    # Remove columns with more than 5% missing values
    remove = df.isnull().sum() > 5/100*len(df)
    remove = remove.index[remove]
    df = df.drop(remove, axis=1)
    
    # Remove text columns with missing values
    remove = df.select_dtypes(include='object').isnull().sum()>0
    remove = remove.index[remove]
    df = df.drop(remove, axis=1)
    
    return  df

len(remove_features(df).columns)

58

In order to select numeric columns most useful for the prediction, we will compute correlation coefficients between 'SalePrice' and the other columns. Numeric columns with low correlation coefficient (a threshold of 0.4 is the default) with the target variable are removed.

In [75]:
def remove_features(df,
                    features_to_remove=["PID", "Order","Mo Sold", "Sale Condition", "Sale Type", "Yr Sold" ],
                    target = 'SalePrice',
                    coeff_threshold=0.4):
    df = df.drop(features_to_remove,axis=1)
    
    # Remove columns with more than 5% missing values
    remove = df.isnull().sum() > 5/100*len(df)
    remove = remove.index[remove]
    df = df.drop(remove, axis=1)
    
    # Remove text columns with missing values
    remove = df.select_dtypes(include='object').isnull().sum()>0
    remove = remove.index[remove]
    df = df.drop(remove, axis=1)
    
    # Remove numeric columns with low correlation coefficient with the target variable
    correlations = df.select_dtypes(exclude='object').corr()[target].abs().sort_values()
    df=df.drop(correlations[correlations < coeff_threshold].index,axis=1)
    
    return  df

len(remove_features(df).columns)

39

Which columns in the data frame should be converted to the categorical data type? All of the columns that can be identified as nominal variables are candidates for being converted to categorical. Based on the [documentation of the data set](https://s3.amazonaws.com/dq-content/307/data_description.txt), the following are nominal features: `PID`, `MS SubClass`, `MS Zoning`, `Street`, `Alley`, `Land Contour`, `Lot Config`, `Neighborhood`, `Condition 1`, `Condition 2`, `Bldg Type`, `House Style`, `Roof Style`, `Roof Matl`, `Exterior 1st`, `Exterior 2nd`, `Mas Vnr Type`, `Foundation`, `Heating`, `Central Air`, `Garage Type`, `Misc Feature`, `Sale Type`, and `Sale Condition`. We will create a function (`convert_to_category()`) that:
- converts these features to the categorical type
- if a categorical column has more than 10 (aribtrary cutoff) unique values, we might drop it, since  we don't want more than 10 columns to be added to the data frame when we dummy code this column
- creates dummy columns and adds them back to the dataframe, while dropping the original text columns

In [76]:
def convert_to_category(df,
                        nominal_features=["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                        "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                        "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                        "Misc Feature", "Sale Type", "Sale Condition"],
                        uniq_threshold=10):
    
    # Select only nominal features present in the data frame
    nominal_features_to_keep = []
    for col in nominal_features:
        if col in df.columns:
            nominal_features_to_keep.append(col)
    
    # Check if the input nominal features correspond to text columns
    df_text_cols = df[nominal_features_to_keep].dtypes == 'object'
    if df_text_cols.all() == False:
        print("Error in `nominal_features` argument: not all of the specified columns are text columns!")
        return df_text_cols
    
    # Remove text columns in the data frame not among the input nominal features
    text_cols_to_remove = []
    for col in df.select_dtypes(include=['object']).columns:
        if col not in nominal_features_to_keep:
            text_cols_to_remove.append(col)
    df = df.drop(text_cols_to_remove, axis=1)
    
    # Drop nominal features with more than `uniq_threshold` categories
    uniqueness_counts = df[nominal_features_to_keep].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > uniq_threshold].index.tolist()
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    # Conversion to categorical type
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    
    # Create dummy columns and add them back to the dataframe, while dropping the original text columns
    new_cols = pd.get_dummies(df.select_dtypes(include=['category']))
    df = pd.concat([df,new_cols], axis=1)
    
    return df

df_subset = remove_features(df)
len(convert_to_category(df_subset).columns)

103

We will now update the `train_and_test()` function in order to incorporate row randomization and cross-validation. Also, we will remove the `features` argument, since these will be selected by the other functions.

In [85]:
from sklearn.model_selection import KFold

In [105]:
def train_and_test(df,
                   target = 'SalePrice',
                   k=0):  
    
    # If k == 1, randomize all rows (frac=1) from `df` once
    if k == 1:
        df = df.sample(frac=1)
    
    # Train and test data split
    train = df[:int(len(df)/2)]
    test = df[int(len(df)/2):]
             
    # Selection of numeric columns
    train = train.select_dtypes(include=['integer', 'float'])
    test = test.select_dtypes(include=['integer', 'float'])
    
    # Model
    features = train.columns.drop(target)
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    
    # If k > 1, iterate over k-folds
    if k > 1:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr = LinearRegression()
            lr.fit(train[features], train[target])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test[target], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
    
    return rmse


df_subset = transform_features(df)
df_subset = remove_features(df_subset)
df_subset = convert_to_category(df_subset)
rmse = train_and_test(df_subset,k=4)
rmse

[29410.06249325091, 38224.87576403876, 28434.03806000115, 29418.745660257788]


31371.930494387154